In [ ]:
%load_ext autoreload
%autoreload 2

from tqdm import tqdm
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from datetime import date
from glob import glob
import os

In [ ]:
START_DATE = '2022-01-01'
END_DATE = '2024-12-31'

COVERAGE_FREQ = 'W'
COVERAGE_FREQ_STRFTIME = '%Y-%W'

TRIP_COUNT_THRESHOLD = 1000
LINK_COUNT_THRESHOLD = 8

In [ ]:
def get_file_name(path):
    name, _ = os.path.splitext(os.path.basename(path))
    return name

def iso_weeks_in_year(yr: int) -> int:
    """
    Number of ISO weeks in calendar year `yr`.
    Dec 31 can fall in week 1, so we treat that as 52.
    """
    w = date(yr, 12, 31).isocalendar()[1]
    return 52 if w == 1 else w

def summarize_trips(path: str) -> pd.DataFrame:
    df_uncleaned = pd.read_parquet(path)
    df_uncleaned['has_geometry'] = df_uncleaned['from_geometry'].notna() & df_uncleaned['to_geometry'].notna()

    df = df_uncleaned[df_uncleaned['valid'] & df_uncleaned['valid_dwell_times'] & df_uncleaned['from_geometry'].notna() & df_uncleaned['to_geometry'].notna()]
    has_geometry_mask = df.groupby(['line', 'route', 'route_id', 'trip', 'date'])['has_geometry'].transform('all')
    df = df.loc[has_geometry_mask]
    df = df.copy()

    if df.empty:
        return pd.DataFrame({
            'lau':                 [get_file_name(path)],
            'pct_valid':           [0],
            'line_pct_valid':     [0]
        })

    # extract year & ISO‑week
    df['year'] = df['date'].dt.year
    df['week'] = df['date'].dt.isocalendar().week.astype(int)

    def calc_cov(gp):
        covs = []
        for yr in (2022, 2023, 2024):
            weeks_seen = gp.loc[gp['year']==yr, 'week'].nunique()
            covs.append(weeks_seen / iso_weeks_in_year(yr))
        return min(covs)  # will be 0 if any year is missing

    lau_coverage = calc_cov(df[['year', 'week']].drop_duplicates())
    
    line_coverage = df[['line', 'year', 'week']].drop_duplicates() \
        .groupby(['line']) \
        .apply(calc_cov) \
        .reset_index(name='line_coverage')

    link_counts = df[['line', 'route', 'route_id']].drop_duplicates()
    link_counts['link_count'] = link_counts['route'].str.count('>') + 1

    trip_counts = df[['date', 'line', 'route', 'route_id', 'trip']].drop_duplicates()
    trip_counts = trip_counts.groupby(['line', 'route', 'route_id']) \
        .apply(lambda g: g.drop_duplicates(subset=['date','trip']).shape[0]) \
        .reset_index(name='trip_count')

    route_info = line_coverage \
        .merge(link_counts, on=['line']) \
        .merge(trip_counts, on=['route', 'route_id']) \

    total_trip_count = df[['date', 'line', 'route', 'route_id', 'trip']].drop_duplicates()
    total_trip_count = total_trip_count.drop_duplicates(subset=['line','date','trip']).shape[0]

    uncleaned_total_trip_count = df_uncleaned[['date', 'line', 'route', 'route_id', 'trip']].drop_duplicates()
    uncleaned_total_trip_count = uncleaned_total_trip_count.drop_duplicates(subset=['line','date','trip']).shape[0]

    route_info['lau'] = get_file_name(path)
    route_info['lau_coverage'] = lau_coverage
    route_info['total_trip_count'] = total_trip_count
    route_info['uncleaned_total_trip_count'] = uncleaned_total_trip_count

    return route_info

paths = glob('/mnt/nvme/sql/kv6_validated/travel_time/*.parquet')
counts = [summarize_trips(p) for p in tqdm(paths)]
summary_df = pd.concat(counts, ignore_index=True)
summary_df

In [ ]:
summary_df.to_parquet('summary_df.parquet')

In [ ]:
summary_df = pd.read_parquet('summary_df.parquet')
summary_df

In [ ]:
nuts = pd.read_csv('/data/dev/benchmark/sampling/EU-27-LAU-2023-NUTS-2021.csv', sep=';')
nuts

In [ ]:
from sqlalchemy import create_engine
engine = create_engine(
    "postgresql+psycopg2://redacted:redacted@db:5432/bison"
)

def get_stratum(row):
    if row['DEGURBA'] == 1 and row['POPULATION'] > 250000:
        return '1.0'
    elif row['DEGURBA'] == 1:
        return '1.1'
    else:
        return str(row['DEGURBA'])
nuts['DEGURBA_EXT'] = nuts.apply(get_stratum, axis=1)

In [ ]:
def route_has_duplicate_stops(row):
    stops = row['route'].split('>')
    for i in range(len(stops) - 1):
        if stops[i] == stops[i + 1]:
            return True
    return False

dummy_stops = pd.read_csv('dummy-blacklist.csv', dtype=str)
dummy_stops = dummy_stops['dataownercode'] + ':' + dummy_stops['userstopcode']

def route_has_dummy_stop(row):
    stops = row['route'].split('>')
    return dummy_stops.isin(stops).any()

tmp = summary_df[
    (summary_df['lau_coverage'] >= 0.5) &
    (summary_df['line_coverage'] >= 0.5) &
    (summary_df['trip_count'] >= 100) &
    (summary_df['link_count'] >= 5)
]
tmp = tmp[
    ~tmp.apply(route_has_dummy_stop, axis=1)
]
tmp['retention_rate'] = tmp.groupby('lau')['trip_count'].transform('sum') / tmp['uncleaned_total_trip_count']
tmp = tmp[tmp['retention_rate'] > 0.25].groupby('lau').agg({'trip_count': 'sum'})

df_merged = pd.merge(
    nuts,
    tmp,
    left_on='LAU CODE',
    right_on='lau',
    how='inner'
)

sampled_laus = []
for degurba, group in df_merged.groupby('DEGURBA_EXT'):
    if degurba == '1.0':
        sampled = group
    else:
        sampled = group.sample(5)
    sampled = sampled.sort_values(['DEGURBA_EXT', 'POPULATION'], ascending=[True, False])
    print(f"Sampled {degurba} ({len(sampled)} out of {len(group)})")
    print(f"Trip count: {sampled['trip_count'].sum()}")
    display(sampled)
    sampled_laus.append(sampled)


In [ ]:
sampled_laus_concat = pd.concat(sampled_laus)
sampled_laus_concat.to_csv('sampled_laus.csv', index=False)
sampled_laus_concat

In [ ]:
sampled_laus_concat = pd.read_csv('sampled_laus.csv')

In [ ]:
def clean_for_output(df):
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    df['from_time'] = df['from_time'].dt.strftime('%Y-%m-%d %H:%M:%S%z')
    df['to_time'] = df['to_time'].dt.strftime('%Y-%m-%d %H:%M:%S%z')
    df['valid'] = (df['valid'] & df['valid_dwell_times']).astype('int8')
    df['route'] = df['route_id']
    if 'from_geometry' in df.columns:
        return df[['lau', 'date', 'line', 'trip', 'route', 'from_stop', 'to_stop', 'from_geometry', 'to_geometry', 'from_time', 'to_time']]
    else:
        return df[['lau', 'date', 'line', 'trip', 'route', 'stop', 'geometry', 'from_time', 'to_time']]

dummy_stops = pd.read_csv('dummy-blacklist.csv', dtype=str)
dummy_stops = dummy_stops['dataownercode'] + ':' + dummy_stops['userstopcode']

def route_has_dummy_stop(row):
    stops = row['route'].split('>')
    return dummy_stops.isin(stops).any()

for lau in sampled_laus_concat['LAU CODE'].drop_duplicates().iloc[::-1]:
    print(f"Processing {lau}")
    routes = summary_df[
        (summary_df['lau'] == lau) &
        (summary_df['lau_coverage'] >= 0.5) &
        (summary_df['line_coverage'] >= 0.50) &
        (summary_df['trip_count'] >= 100) &
        (summary_df['link_count'] >= 5)
    ]
    routes = routes[~routes.apply(route_has_dummy_stop, axis=1)]
    routes = routes[['route', 'route_id']].drop_duplicates()
    #display(routes)

    print(f"Processing travel time for {lau}")
    tt = pd.read_parquet(f'/mnt/nvme/sql/kv6_validated/travel_time/{lau}.parquet')
    tt['has_geometry'] = tt['from_geometry'].notna() & tt['to_geometry'].notna()
    tt = tt[tt['valid'] & tt['valid_dwell_times']]
    has_geometry_mask = tt.groupby(['line', 'route', 'route_id', 'trip', 'date'])['has_geometry'].transform('all')
    tt = tt.loc[has_geometry_mask]
    tt = pd.merge(
        tt,
        routes,
        on=['route', 'route_id'],
        how='inner'
    )
    tt = clean_for_output(tt)
    tt.to_csv(f'/mnt/nvme/sql/kv6_validated/travel_time_filtered/{lau}.csv', index=False)
    
    print(f"Processing dwell time for {lau}")
    dt = pd.read_parquet(f'/mnt/nvme/sql/kv6_validated/dwell_time/{lau}.parquet')
    dt['has_geometry'] = dt['geometry'].notna()
    # not sure if removing all the ones with no route is correct
    dt = dt[dt['valid'] & dt['valid_dwell_times'] & dt['route_id'].notna()]
    # does this filter out the same ones as above? probably should match trips instead?
    has_geometry_mask = dt.groupby(['line', 'route', 'route_id', 'trip', 'date'])['has_geometry'].transform('all')
    dt = dt.loc[has_geometry_mask]
    dt = pd.merge(
        dt,
        routes,
        on=['route', 'route_id'],
        how='inner'
    )
    dt = clean_for_output(dt)
    dt.to_csv(f'/mnt/nvme/sql/kv6_validated/dwell_time_filtered/{lau}.csv', index=False)